# Extending the Library – Practical Example

This notebook provides a **practical walkthrough** of how to extend the library by building
a simple **text cleaning pipeline**.

We will cover:
- creating a custom Footprint
- defining Steps (functions and classes)
- composing and executing a Trail
- inspecting execution metadata

This notebook complements the theoretical documentation in:

📄 [`docs/extending.md`](../docs/extending.md)

## Setup

We begin by installing the library directly from the GitHub repository.

The example that follows focuses on **clarity and readability**, rather than performance or completeness.

In [11]:
pip install "git+https://github.com/emanuelegaliano/TRACCIA.git" # type: ignore

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emanuelegaliano/TRACCIA.git to /tmp/pip-req-build-ptvecwge
  Running command git clone --filter=blob:none --quiet https://github.com/emanuelegaliano/TRACCIA.git /tmp/pip-req-build-ptvecwge
  Resolved https://github.com/emanuelegaliano/TRACCIA.git to commit 4ab996160108be1ca6e76d3457e0ca66ed5eb04e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


With the library installed, we can now import the core building blocks required to define our pipeline.

These components will be used to model the shared state, define processing steps, and compose the execution flow.

In [1]:
from traccia import (
    FootprintMetadata,
    step,
    Step,
    Trail,
)

## 1. Defining a Custom Footprint

The Footprint represents the **shared mutable state** that flows through the pipeline.

In this example, the footprint will:
- hold a text string
- expose execution metadata via `get_metadata()`

⚠️ Important:
The metadata instance **must always be the same** for the lifetime of the footprint.

In [2]:
class TextFootprint:
    def __init__(self, text: str):
        self.text = text
        self._metadata = FootprintMetadata()

    def get_metadata(self) -> FootprintMetadata:
        return self._metadata

Let’s instantiate the footprint and inspect its initial state.

In [3]:
fp = TextFootprint("  Hello,   WORLD!!  ")
fp.text

'  Hello,   WORLD!!  '

## 2. Creating Steps as Functions

The recommended way to define steps is by using the `@step()` decorator.

Each step:
- receives the footprint
- mutates it
- returns it

Steps should be **small and single-purpose**.

In [13]:
@step()
def normalize_whitespace(fp: TextFootprint) -> TextFootprint:
    fp.text = " ".join(fp.text.split())
    return fp

In [5]:
@step()
def to_lowercase(fp: TextFootprint) -> TextFootprint:
    fp.text = fp.text.lower()
    return fp

## 3. Creating Steps as Classes

Class-based steps are useful when:
- configuration is required
- state or dependencies are needed

They must be **callable** and respect the same step contract.

In [6]:
class RemoveCharacters:
    def __init__(self, chars: str):
        self.chars = chars

    def __call__(self, fp: TextFootprint) -> TextFootprint:
        for c in self.chars:
            fp.text = fp.text.replace(c, "")
        return fp

In [7]:
remove_punctuation = Step(
    name="remove_punctuation",
    fn=RemoveCharacters(chars=".,!?")
)

## 4. Composing a Trail

A `Trail` defines the execution order of steps.

The API is fluent and encourages readable pipeline definitions.


In [8]:
trail = (
    Trail()
    .then(normalize_whitespace)
    .then(to_lowercase)
    .then(remove_punctuation)
)

## 5. Executing the Pipeline

We now run the trail on our footprint.


In [9]:
result = trail.run(fp)
result.text

'hello world'

## 6. Inspecting Execution Metadata

The metadata attached to the footprint records information about execution.

This is useful for:
- tracing
- debugging
- observability

In [10]:
metadata = result.get_metadata()
metadata

FootprintMetadata(run_id=None, created_at=datetime.datetime(2025, 12, 21, 21, 35, 18, 224285, tzinfo=datetime.timezone.utc), started_at=datetime.datetime(2025, 12, 21, 21, 36, 58, 3282, tzinfo=datetime.timezone.utc), finished_at=datetime.datetime(2025, 12, 21, 21, 36, 58, 3295, tzinfo=datetime.timezone.utc), handlers=['normalize_whitespace', 'to_lowercase', 'remove_punctuation'], tags={'trail': 'Trail'}, extra={})

## 7. Modifying a Trail Dynamically

Trails are mutable by design.

This allows inserting, replacing, or removing steps without redefining the pipeline.

In [17]:
trail.insert_before(
    "to_lowercase", 
    normalize_whitespace
    )

Trail(steps=[Step(name='normalize_whitespace', fn=<function normalize_whitespace at 0x7ff88bc4c670>), Step(name='normalize_whitespace', fn=<function normalize_whitespace at 0x7ff869518b40>), Step(name='normalize_whitespace', fn=<function normalize_whitespace at 0x7ff869518b40>), Step(name='normalize_whitespace', fn=<function normalize_whitespace at 0x7ff869518b40>), Step(name='normalize_whitespace', fn=<function normalize_whitespace at 0x7ff869518b40>), Step(name='to_lowercase', fn=<function to_lowercase at 0x7ff88bc4ca90>), Step(name='remove_punctuation', fn=<__main__.RemoveCharacters object at 0x7ff88bbeb8c0>)], name='Trail', run_id=None, default_tags={}, trace_enabled=False)

Dynamic trail composition enables:
- plugin systems
- environment-specific behavior
- optional features

All without copying or rewriting pipelines.

## 8. Testing the Extension
If you want to contribute to the library, tests should ship with the code, as explained in [docs/contribute.md](../docs/contribute.md)

In this section we outline:
- where extension tests should live
- what to test (unit vs integration)
- how to run the full test suite with pytest


### Example tests (using the objects defined above)

The following cells show **small pytest-style tests** using the exact `TextFootprint`, steps,
and `Trail` defined earlier in this notebook.

These are meant as examples: in a real extension, you would move each test into
`tests/<extension_name>/test_*.py` and keep the same assertions.

In [18]:
def test_normalize_whitespace_collapses_spaces():
    fp = TextFootprint("  Hello,   WORLD!!  ")
    out = normalize_whitespace(fp)
    assert out.text == "Hello, WORLD!!"


def test_to_lowercase_converts_to_lowercase():
    fp = TextFootprint("Hello WORLD")
    out = to_lowercase(fp)
    assert out.text == "hello world"


def test_remove_punctuation_step_removes_chars():
    fp = TextFootprint("Hello, WORLD!!")
    out = remove_punctuation(fp)
    assert out.text == "Hello WORLD"


def test_trail_end_to_end_text_cleaning():
    fp = TextFootprint("  Hello,   WORLD!!  ")
    out = trail.run(fp)
    assert out.text == "hello world"


# Execute the example tests in the notebook
test_normalize_whitespace_collapses_spaces()
test_to_lowercase_converts_to_lowercase()
test_remove_punctuation_step_removes_chars()
test_trail_end_to_end_text_cleaning()

print("✅ Example tests passed")

✅ Example tests passed


### Note on Real-World Usage

The tests shown above are **demonstration examples only**.

In a real contribution, tests should:
- live exclusively under the `tests/` directory
- import functions and classes from the extension modules (e.g. `routes/<extension_name>/...`)
- follow the structure and guidelines described in [`docs/contribute.md`](../docs/contribute.md)

Notebook-based tests are meant to illustrate behavior, not to replace the project test suite.